In [1]:
import keras
import theano
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from keras.models import Sequential
from keras.layers.core import Dense, Activation

Using Theano backend.


In [2]:
try:
    with open('./input/inputs.npy',"rb") as f:
        inputs = np.load(f)
    with open('./input/outputs.npy',"rb") as f:
        outputs = np.load(f)
        
except:
    data = np.genfromtxt('./input/train.csv', delimiter=',',skip_header=1,dtype=np.uint8)
    inputs = data[:,1:]
    outputs = data[:,0]
    np.save('./input/inputs.npy',inputs)
    np.save('./input/outputs.npy',outputs)
print(f'Input:\nShape:{inputs.shape}\n{inputs[:5,:5]}\n')
print(f'Output:\nShape:{outputs.shape}\n{outputs[:5]}\n')

Input:
Shape:(42000, 784)
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

Output:
Shape:(42000,)
[1 0 1 4 0]



In [3]:
def onehot(ARRAY):
    return np.eye(10)[ARRAY].astype(np.uint8)

In [4]:
inputs = normalize(inputs)
X_train, X_test, y_train, y_test = train_test_split(inputs,outputs,test_size = 0.2)

X_train = X_train
y_train_hot = onehot(y_train)

print(X_train.shape)
print(y_train_hot.shape)

(33600, 784)
(33600, 10)


In [5]:
model = Sequential()

model.add(Dense(input_dim=784, units=350))
model.add(Activation("sigmoid"))
model.add(Dense(units=10))
model.add(Activation("softmax"))

In [6]:
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

model.fit(X_train, y_train_hot, epochs=12, batch_size=336)

model.evaluate(X_test, onehot(y_test), batch_size=840)

Epoch 1/12
33600/33600 [==============================] - 23s 680us/step - loss: 1.8887 - acc: 0.548917s
Epoch 2/12
33600/33600 [==============================] - 23s 679us/step - loss: 1.1014 - acc: 0.8048
Epoch 3/12
33600/33600 [==============================] - 23s 679us/step - loss: 0.7024 - acc: 0.8513
Epoch 4/12
33600/33600 [==============================] - 23s 683us/step - loss: 0.5332 - acc: 0.87482s - loss: 0.5366 - a
Epoch 5/12
33600/33600 [==============================] - 23s 681us/step - loss: 0.4457 - acc: 0.8878
Epoch 6/12
33600/33600 [==============================] - 23s 687us/step - loss: 0.3955 - acc: 0.89638s - loss: 0.4016 - acc:  - ETA: 7s - loss: 0.3975 - acc: 0. - ETA: 6s
Epoch 7/12
33600/33600 [==============================] - 23s 698us/step - loss: 0.3625 - acc: 0.90233s - loss: 0.3
Epoch 8/12
33600/33600 [==============================] - 23s 691us/step - loss: 0.3403 - acc: 0.9059
Epoch 9/12
33600/33600 [==============================] - 23s 689us/step - l

[0.31120160639286043, 0.9125]